In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# Notebook is saved under /notebooks, so project root is one level up
PROJECT_ROOT = Path("..").resolve()
RAW_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

# Display options for easier inspection
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

print("Project root:", PROJECT_ROOT)
print("Raw dir:", RAW_DIR)
print("Processed dir:", PROCESSED_DIR)
print("pandas version:", pd.__version__)

Project root: C:\LABS\Week8\Mezcal-final
Raw dir: C:\LABS\Week8\Mezcal-final\data\raw
Processed dir: C:\LABS\Week8\Mezcal-final\data\processed
pandas version: 2.3.1


In [4]:
prod_path = RAW_DIR / "comercam" / "mezcal_master_production_2005_2024.csv"
df_prod = pd.read_csv(prod_path)

print("Shape:", df_prod.shape)
print("Columns:", list(df_prod.columns))
display(df_prod.head())
display(df_prod.dtypes)

Shape: (14, 3)
Columns: ['Unnamed: 0', 'Year', 'Production_liters']


,Unnamed: 0,Year,Production_liters
0,0,2011.0,980375.0
1,1,2012.0,1044696.0
2,2,2013.0,2519568.0
3,3,2014.0,1451718.0
4,4,2015.0,2419175.0


Unnamed: 0             int64
Year                 float64
Production_liters    float64
dtype: object

In [5]:
def csv_mini_clean(df, numeric_cols=None, date_cols=None, dropna_subset=None):
    df2 = df.copy()
    obj_cols = df2.select_dtypes(include="object").columns
    for c in obj_cols:
        df2[c] = df2[c].str.strip()
    if numeric_cols:
        for c in numeric_cols:
            df2[c] = pd.to_numeric(df2[c], errors="coerce")
    if date_cols:
        for c in date_cols:
            df2[c] = pd.to_datetime(df2[c], errors="coerce")
    df2 = df2.loc[:, ~df2.columns.str.match(r"^Unnamed")]
    df2 = df2.drop_duplicates()
    if dropna_subset:
        df2 = df2.dropna(subset=dropna_subset)
    return df2

In [6]:
clean_prod = csv_mini_clean(
    df_prod,
    numeric_cols=["Production_liters"],
    dropna_subset=["Year", "Production_liters"]
)
clean_prod["Year"] = pd.to_numeric(clean_prod["Year"], errors="coerce").astype("Int64")

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "production_2005_2024.csv"
clean_prod.to_csv(out_path, index=False)

display(clean_prod.head())
display(clean_prod.dtypes)
print(out_path)

,Year,Production_liters
0,2011,980375.0
1,2012,1044696.0
2,2013,2519568.0
3,2014,1451718.0
4,2015,2419175.0


Year                   Int64
Production_liters    float64
dtype: object

C:\LABS\Week8\Mezcal-final\data\processed\production_2005_2024.csv


In [7]:
paths = {
    "export": RAW_DIR / "comercam" / "mezcal_master_bottling_export_2011_2024.csv",
    "national": RAW_DIR / "comercam" / "mezcal_master_bottling_national_2011_2024.csv",
    "indicators": RAW_DIR / "comercam" / "mezcal_master_indicators_2021_2024.csv",
}

for name, p in paths.items():
    df = pd.read_csv(p)
    print(f"\n>>> {name.upper()} — {p.name}")
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))
    display(df.head())


>>> EXPORT — mezcal_master_bottling_export_2011_2024.csv
Shape: (14, 3)
Columns: ['Unnamed: 0', 'Year', 'Liters']


,Unnamed: 0,Year,Liters
0,0,2011,647989
1,1,2012,768014
2,2,2013,914317
3,3,2014,1157420
4,4,2015,1480263



>>> NATIONAL — mezcal_master_bottling_national_2011_2024.csv
Shape: (14, 3)
Columns: ['Unnamed: 0', 'Year', 'Liters']


,Unnamed: 0,Year,Liters
0,0,2011,270426
1,1,2012,317803
2,2,2013,800002
3,3,2014,924686
4,4,2015,1508839



>>> INDICATORS — mezcal_master_indicators_2021_2024.csv
Shape: (37, 3)
Columns: ['Year', 'Indicator', 'Value']


,Year,Indicator,Value
0,2021,Production_liters,8099591.0
1,2021,Bottling_national,3684458.0
2,2021,Bottling_export,5102520.0
3,2021,Registered_properties,11531.0
4,2021,Registered_associates,3960.0


In [8]:
path = RAW_DIR / "comercam" / "mezcal_master_bottling_export_2011_2024.csv"
df = pd.read_csv(path)

def csv_mini_clean(df, numeric_cols=None, date_cols=None, dropna_subset=None):
    df2 = df.copy()
    obj_cols = df2.select_dtypes(include="object").columns
    for c in obj_cols:
        df2[c] = df2[c].str.strip()
    if numeric_cols:
        for c in numeric_cols:
            df2[c] = pd.to_numeric(df2[c], errors="coerce")
    if date_cols:
        for c in date_cols:
            df2[c] = pd.to_datetime(df2[c], errors="coerce")
    df2 = df2.loc[:, ~df2.columns.str.match(r"^Unnamed")]
    df2 = df2.drop_duplicates()
    if dropna_subset:
        df2 = df2.dropna(subset=dropna_subset)
    return df2

clean_export = csv_mini_clean(
    df, numeric_cols=["Liters"], dropna_subset=["Year", "Liters"]
)
clean_export["Year"] = pd.to_numeric(clean_export["Year"], errors="coerce").astype("Int64")

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "bottling_export_2011_2024.csv"
clean_export.to_csv(out_path, index=False)

display(clean_export.head())
display(clean_export.dtypes)
print(out_path)

,Year,Liters
0,2011,647989
1,2012,768014
2,2013,914317
3,2014,1157420
4,2015,1480263


Year      Int64
Liters    int64
dtype: object

C:\LABS\Week8\Mezcal-final\data\processed\bottling_export_2011_2024.csv


In [9]:
import pandas as pd

path = RAW_DIR / "comercam" / "mezcal_master_bottling_national_2011_2024.csv"
df = pd.read_csv(path)

def csv_mini_clean(df, numeric_cols=None, date_cols=None, dropna_subset=None):
    df2 = df.copy()
    obj_cols = df2.select_dtypes(include="object").columns
    for c in obj_cols:
        df2[c] = df2[c].str.strip()
    if numeric_cols:
        for c in numeric_cols:
            df2[c] = pd.to_numeric(df2[c], errors="coerce")
    if date_cols:
        for c in date_cols:
            df2[c] = pd.to_datetime(df2[c], errors="coerce")
    df2 = df2.loc[:, ~df2.columns.str.match(r"^Unnamed")]
    df2 = df2.drop_duplicates()
    if dropna_subset:
        df2 = df2.dropna(subset=dropna_subset)
    return df2

clean_nat = csv_mini_clean(df, numeric_cols=["Liters"], dropna_subset=["Year","Liters"])
clean_nat["Year"] = pd.to_numeric(clean_nat["Year"], errors="coerce").astype("Int64")

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "bottling_national_2011_2024.csv"
clean_nat.to_csv(out_path, index=False)

display(clean_nat.head())
display(clean_nat.dtypes)
print(out_path)

,Year,Liters
0,2011,270426
1,2012,317803
2,2013,800002
3,2014,924686
4,2015,1508839


Year      Int64
Liters    int64
dtype: object

C:\LABS\Week8\Mezcal-final\data\processed\bottling_national_2011_2024.csv


In [10]:
path = RAW_DIR / "comercam" / "mezcal_master_indicators_2021_2024.csv"
df = pd.read_csv(path)

def csv_mini_clean(df, numeric_cols=None, date_cols=None, dropna_subset=None):
    df2 = df.copy()
    obj_cols = df2.select_dtypes(include="object").columns
    for c in obj_cols:
        df2[c] = df2[c].str.strip()
    if numeric_cols:
        for c in numeric_cols:
            df2[c] = pd.to_numeric(df2[c], errors="coerce")
    if date_cols:
        for c in date_cols:
            df2[c] = pd.to_datetime(df2[c], errors="coerce")
    df2 = df2.loc[:, ~df2.columns.str.match(r"^Unnamed")]
    df2 = df2.drop_duplicates()
    if dropna_subset:
        df2 = df2.dropna(subset=dropna_subset)
    return df2

clean_ind = csv_mini_clean(
    df, numeric_cols=["Value"], dropna_subset=["Year","Indicator","Value"]
)
clean_ind["Year"] = pd.to_numeric(clean_ind["Year"], errors="coerce").astype("Int64")
clean_ind["Indicator"] = clean_ind["Indicator"].astype("string")

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "indicators_2021_2024.csv"
clean_ind.to_csv(out_path, index=False)

display(clean_ind.head())
display(clean_ind.dtypes)
print(out_path)

,Year,Indicator,Value
0,2021,Production_liters,8099591.0
1,2021,Bottling_national,3684458.0
2,2021,Bottling_export,5102520.0
3,2021,Registered_properties,11531.0
4,2021,Registered_associates,3960.0


Year                  Int64
Indicator    string[python]
Value               float64
dtype: object

C:\LABS\Week8\Mezcal-final\data\processed\indicators_2021_2024.csv


In [11]:
pp = PROCESSED_DIR

prod = pd.read_csv(pp / "production_2005_2024.csv")
exp  = pd.read_csv(pp / "bottling_export_2011_2024.csv")
nat  = pd.read_csv(pp / "bottling_national_2011_2024.csv")
ind  = pd.read_csv(pp / "indicators_2021_2024.csv")

def quick_report(name, df, val_col):
    print(f"\n== {name} ==")
    print("shape:", df.shape)
    print("years:", int(df["Year"].min()), "→", int(df["Year"].max()))
    print("nulls:", df.isnull().sum().to_dict())
    print("negatives in", val_col, ":", int((df[val_col] < 0).sum()) if val_col in df.columns else "n/a")
    print("dtypes:", df.dtypes.to_dict())

quick_report("PRODUCTION", prod, "Production_liters")
quick_report("EXPORT", exp, "Liters")
quick_report("NATIONAL", nat, "Liters")
quick_report("INDICATORS", ind, "Value")

common_years = sorted(set(exp["Year"].dropna().astype(int)).intersection(set(nat["Year"].dropna().astype(int))))
print("\nCommon years export vs national:", common_years[:3], "...", common_years[-3:], "(n=", len(common_years), ")")


== PRODUCTION ==
shape: (14, 2)
years: 2011 → 2024
nulls: {'Year': 0, 'Production_liters': 0}
negatives in Production_liters : 0
dtypes: {'Year': dtype('int64'), 'Production_liters': dtype('float64')}

== EXPORT ==
shape: (14, 2)
years: 2011 → 2024
nulls: {'Year': 0, 'Liters': 0}
negatives in Liters : 0
dtypes: {'Year': dtype('int64'), 'Liters': dtype('int64')}

== NATIONAL ==
shape: (14, 2)
years: 2011 → 2024
nulls: {'Year': 0, 'Liters': 0}
negatives in Liters : 0
dtypes: {'Year': dtype('int64'), 'Liters': dtype('int64')}

== INDICATORS ==
shape: (37, 3)
years: 2021 → 2024
nulls: {'Year': 0, 'Indicator': 0, 'Value': 0}
negatives in Value : 0
dtypes: {'Year': dtype('int64'), 'Indicator': dtype('O'), 'Value': dtype('float64')}

Common years export vs national: [2011, 2012, 2013] ... [2022, 2023, 2024] (n= 14 )


In [12]:
pp = PROCESSED_DIR

prod = pd.read_csv(pp / "production_2005_2024.csv")
exp  = pd.read_csv(pp / "bottling_export_2011_2024.csv")
nat  = pd.read_csv(pp / "bottling_national_2011_2024.csv")

for df in (prod, exp, nat):
    df["Year"] = df["Year"].astype(int)

def add_yoy(df, value_col):
    df = df.sort_values("Year").copy()
    df["YoY_%"] = df[value_col].pct_change() * 100
    return df

def cagr(df, value_col):
    d = df.sort_values("Year").dropna(subset=[value_col])
    y0, y1 = int(d["Year"].iloc[0]), int(d["Year"].iloc[-1])
    n = y1 - y0
    v0, v1 = float(d[value_col].iloc[0]), float(d[value_col].iloc[-1])
    return np.nan if n <= 0 or v0 <= 0 else (v1 / v0) ** (1 / n) - 1

prod_r = add_yoy(prod, "Production_liters")
exp_r  = add_yoy(exp,  "Liters")
nat_r  = add_yoy(nat,  "Liters")

prod_cagr = cagr(prod_r, "Production_liters")
exp_cagr  = cagr(exp_r,  "Liters")
nat_cagr  = cagr(nat_r,  "Liters")

bot = exp.merge(nat, on="Year", suffixes=("_export","_national"))
bot["Total_bottling"]   = bot["Liters_export"] + bot["Liters_national"]
bot["Export_share_%"]   = bot["Liters_export"] / bot["Total_bottling"] * 100

summary = pd.DataFrame({
    "Series": ["Production", "Bottling Export", "Bottling National"],
    "Start year": [prod_r["Year"].min(), exp_r["Year"].min(), nat_r["Year"].min()],
    "End year":   [prod_r["Year"].max(), exp_r["Year"].max(), nat_r["Year"].max()],
    "Start value": [prod_r.iloc[0]["Production_liters"], exp_r.iloc[0]["Liters"], nat_r.iloc[0]["Liters"]],
    "End value":   [prod_r.iloc[-1]["Production_liters"], exp_r.iloc[-1]["Liters"], nat_r.iloc[-1]["Liters"]],
    "CAGR_%": [None if pd.isna(prod_cagr) else prod_cagr*100,
               None if pd.isna(exp_cagr)  else exp_cagr*100,
               None if pd.isna(nat_cagr)  else nat_cagr*100]
})

print("=== SUMMARY ===")
display(summary.round(2))

print("\n=== Last 3 years — YoY % ===")
display(prod_r[["Year","YoY_%"]].tail(3).assign(Series="Production"))
display(exp_r[["Year","YoY_%"]].tail(3).assign(Series="Export"))
display(nat_r[["Year","YoY_%"]].tail(3).assign(Series="National"))

print("\n=== Export share of total bottling (last 5 years) ===")
display(bot.sort_values("Year")[["Year","Export_share_%"]].tail(5).round(2))

=== SUMMARY ===


,Series,Start year,End year,Start value,End value,CAGR_%
0,Production,2011,2024,980375.0,11362436.0,20.74
1,Bottling Export,2011,2024,647989.0,7574530.0,20.82
2,Bottling National,2011,2024,270426.0,3793310.0,22.53



=== Last 3 years — YoY % ===


,Year,YoY_%,Series
11,2022,74.891609,Production
12,2023,-13.595350,Production
13,2024,-7.167024,Production


,Year,YoY_%,Series
11,2022,72.208595,Export
12,2023,-21.340420,Export
13,2024,9.588385,Export


,Year,YoY_%,Series
11,2022,138.487669,National
12,2023,-39.366367,National
13,2024,-28.802424,National



=== Export share of total bottling (last 5 years) ===


,Year,Export_share_%
9,2020,66.58
10,2021,58.07
11,2022,50.00
12,2023,56.47
13,2024,66.63


In [18]:
# Clean Interciencia - Economic impact
path = RAW_DIR / "interciencia_magazine" / "mezcal_brands_export_growth_2011_2024.csv"
df = pd.read_csv(path)

def csv_mini_clean(df, numeric_cols=None, date_cols=None, dropna_subset=None):
    df2 = df.copy()
    obj_cols = df2.select_dtypes(include="object").columns
    for c in obj_cols: df2[c] = df2[c].str.strip()
    if numeric_cols:
        for c in numeric_cols: df2[c] = pd.to_numeric(df2[c], errors="coerce")
    if date_cols:
        for c in date_cols: df2[c] = pd.to_datetime(df2[c], errors="coerce")
    df2 = df2.loc[:, ~df2.columns.str.match(r"^Unnamed")]
    df2 = df2.drop_duplicates()
    if dropna_subset: df2 = df2.dropna(subset=dropna_subset)
    return df2

clean_brands = csv_mini_clean(df, numeric_cols=None, dropna_subset=["Year"])
if "Brands" in clean_brands.columns:
    clean_brands["Brands"] = pd.to_numeric(clean_brands["Brands"], errors="coerce").astype("Int64")
if "Export_liters" in clean_brands.columns:
    clean_brands["Export_liters"] = pd.to_numeric(clean_brands["Export_liters"], errors="coerce")

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "brands_export_growth_2011_2024.csv"
clean_brands.to_csv(out_path, index=False)

display(clean_brands.head())
display(clean_brands.dtypes)
print(out_path)

,Year,Export_brands
0,2011,68
1,2024,408


Year             int64
Export_brands    int64
dtype: object

C:\LABS\Week8\Mezcal-final\data\processed\brands_export_growth_2011_2024.csv


In [21]:
path = RAW_DIR / "interciencia_magazine" / "mezcal_economic_impact_2010_2023.csv"
df_raw = pd.read_csv(path)
print(list(df_raw.columns))
display(df_raw.head())

['Period', 'Production_liters', 'Economic_value_million_pesos', 'Direct_employment', 'Indirect_employment']


,Period,Production_liters,Economic_value_million_pesos,Direct_employment,Indirect_employment
0,2010-2023,12239655,36162,55000,210000


In [23]:
# Interciencia Economic impact 
path = RAW_DIR / "interciencia_magazine" / "mezcal_economic_impact_2010_2023.csv"
df = pd.read_csv(path)

df.columns = [c.strip().replace(" ", "_") for c in df.columns]
num_cols = [c for c in df.columns if c != "Period"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

clean_impact_wide = df[["Period"] + num_cols].dropna(how="all").drop_duplicates()

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "economic_impact_2010_2023_wide.csv"
clean_impact_wide.to_csv(out_path, index=False)

display(clean_impact_wide)
display(clean_impact_wide.dtypes)
print(out_path)

,Period,Production_liters,Economic_value_million_pesos,Direct_employment,Indirect_employment
0,2010-2023,12239655,36162,55000,210000


Period                          object
Production_liters                int64
Economic_value_million_pesos     int64
Direct_employment                int64
Indirect_employment              int64
dtype: object

C:\LABS\Week8\Mezcal-final\data\processed\economic_impact_2010_2023_wide.csv


In [25]:
p = RAW_DIR / "interciencia_magazine" / "mezcal_sme_strategies.csv"
d = pd.read_csv(p)

print("shape:", d.shape)
print("columns:", list(d.columns))
print(d.dtypes)
display(d.head(10))

shape: (4, 2)
columns: ['Strategy', 'Mainly_applied_by']
Strategy             object
Mainly_applied_by    object
dtype: object


,Strategy,Mainly_applied_by
0,Low_cost_leadership,Bottlers
1,Differentiation,Producers
2,Customer_service,Both
3,Product_presentation,Both


In [26]:
# Interciencia SME strategies
p = RAW_DIR / "interciencia_magazine" / "mezcal_sme_strategies.csv"
df = pd.read_csv(p)

df.columns = [c.strip().replace(" ", "_") for c in df.columns]
if "Strategy" not in df.columns:
    for c in df.columns:
        if c.lower() in ("strategy","strategies","estrategia","estrategias"):
            df = df.rename(columns={c: "Strategy"})
            break
if "Mainly_applied_by" not in df.columns:
    for c in df.columns:
        if c.lower().replace(" ", "_") in ("mainly_applied_by","aplicada_principalmente_por"):
            df = df.rename(columns={c: "Mainly_applied_by"})
            break

clean_sme = df[["Strategy","Mainly_applied_by"]].dropna(how="any").drop_duplicates()
clean_sme["Strategy"] = clean_sme["Strategy"].astype("string").str.strip()
clean_sme["Mainly_applied_by"] = (
    clean_sme["Mainly_applied_by"]
    .astype("string").str.strip().str.lower()
    .map({"bottlers":"Bottlers","producers":"Producers","both":"Both"})
    .fillna(clean_sme["Mainly_applied_by"].str.title())
)

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "sme_strategies.csv"
clean_sme.to_csv(out_path, index=False)

display(clean_sme)
display(clean_sme.dtypes)
print(out_path)

,Strategy,Mainly_applied_by
0,Low_cost_leadership,Bottlers
1,Differentiation,Producers
2,Customer_service,Both
3,Product_presentation,Both


Strategy             string[python]
Mainly_applied_by            object
dtype: object

C:\LABS\Week8\Mezcal-final\data\processed\sme_strategies.csv


In [20]:
# Databridge projections
path = RAW_DIR / "databridge_market_research" / "europe_mezcal_market_projections.csv"
df = pd.read_csv(path)

df2 = df.copy()
df2.columns = [c.strip().replace(" ", "_") for c in df2.columns]

if "Year" not in df2.columns:
    for c in df2.columns:
        if "year" in c.lower():
            df2 = df2.rename(columns={c: "Year"})
            break

df2["Year"] = pd.to_numeric(df2["Year"], errors="coerce").astype("Int64")

num_cols = [c for c in df2.columns if c != "Year" and pd.api.types.is_numeric_dtype(df2[c])]
if not num_cols:
    for c in [c for c in df2.columns if c != "Year"]:
        df2[c] = pd.to_numeric(df2[c], errors="coerce")
    num_cols = [c for c in df2.columns if c != "Year" and pd.api.types.is_numeric_dtype(df2[c])]

forecast_col = num_cols[0]
clean_proj = df2[["Year", forecast_col]].dropna().drop_duplicates().rename(columns={forecast_col: "EU_market_projection"})

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "eu_market_projection.csv"
clean_proj.to_csv(out_path, index=False)
print(out_path)

C:\LABS\Week8\Mezcal-final\data\processed\eu_market_projection.csv


In [27]:
display(pd.read_csv(PROCESSED_DIR / "eu_market_projection.csv").head())

,Year,EU_market_projection
0,2022,2029
1,2025,2035
2,2024,2030
3,2022,2030


In [28]:
sorted(p.name for p in PROCESSED_DIR.glob("*.csv"))

['bottling_export_2011_2024.csv',
 'bottling_national_2011_2024.csv',
 'brands_export_growth_2011_2024.csv',
 'economic_impact_2010_2023_wide.csv',
 'eu_market_projection.csv',
 'indicators_2021_2024.csv',
 'production_2005_2024.csv',
 'sme_strategies.csv']